In [3]:
import pandas as pd
import requests
import json
import numpy as np
from datetime import datetime
from datetime import date

## Get raw data from [map](https://sfgov.maps.arcgis.com/apps/webappviewer/index.html?id=bb080a525416426c9f96057a00367b4d)

id = bb080a525416426c9f96057a00367b4d

Dataset at https://services.arcgis.com/Zs2aNLFN00jrS4gG/ArcGIS/rest/services/Food_Provider_Resources/FeatureServer/

As of 8/23/2020, last edited 8/23/2020.

In [5]:
url = "https://services.arcgis.com/Zs2aNLFN00jrS4gG/ArcGIS/rest/services/Food_Provider_Resources/FeatureServer/0/query"

params = {
    "f":"json",
    "where":"1=1",
    "returnGeometry":"true",
    "outFields":"*",
}

res = requests.get(url, params=params)
df = pd.json_normalize(json.loads(res.text)['features'])

In [6]:
df.head(10)

,attributes.OBJECTID,attributes.Site,attributes.Operational_Status,attributes.Resource_Type,attributes.Resource_Detail,attributes.Distribution_Method,attributes.Population_Served,attributes.Eligibility_Criteria,attributes.Address,attributes.Days_of_Week,...,attributes.Phone,attributes.Website,attributes.Provider,attributes.Additional_Notes,attributes.Date_updated,attributes.Zip_Code,attributes.Description,attributes.How_to_Access,geometry.x,geometry.y
0,1,30th Street Senior Center,Open,Prepared Meals (Free or Low-Cost),Chilled meal (single),Pickup and Delivery,Seniors,Age 60+,225 30th St,Monday - Saturday,...,(415) 550-2210,None,On Lok Day Services,"Cuisine Type: American, Vegetariana",1.589501e+12,94131.0,"Community dining programs, funded by the Depar...",Seniors (age 60+) can visit an individual site...,-122.424984,37.741897
1,2,A.P Giannini Middle School,Open,Prepared Meals (Free or Low-Cost),Grab-and-go bags with multiple meals,Pickup,Children and Youth,Age 18 and younger,3151 Ortega St,Wednesday,...,None,http://sfusd.edu/schoolfood,None,"Meal bags include breakfast, lunch, supper, fr...",1.593562e+12,94122.0,"During the COVID19 school closure, San Francis...",Families can visit a site during the specified...,-122.496874,37.750832
2,3,Amazon Fresh,Open,Groceries (Purchase),Grocery Bag,Delivery,All San Franciscans,All San Franciscans,Citywide,Daily,...,None,www.amazon fresh.coy,None,None,NaN,Citywide,Meals and groceries from many San Francisco re...,Place order using a smartphone app or web site...,NaN,NaN
3,4,Bayview Opera House - Transitional Pop-Up,Closed to public,Groceries (Free or Low-Cost),Pop-up Pantry,Pickup,Low-income San Franciscans,Low-Income,4705 Third St,Monday,...,(415) 282-1900,http://FoodLocator.SFMFoodBank.org,SF Marin Food Bank,None,1.590451e+12,94124,"Food pantry programs distribute fresh produce,...",Regular/pre-existing sites require enrollment ...,-122.390172,37.735004
4,5,Bernal Heights Neighborhood,Closed to public,Groceries (Free or Low-Cost),Food Pantry,Pickup,Low-income San Franciscans,Low-Income,515 Cortland Ave,Monday - Friday,...,(415) 282-1900,http://FoodLocator.SFMFoodBank.org,SF Marin Food Bank,None,1.589933e+12,94110,"Food pantry programs distribute fresh produce,...",Regular/pre-existing sites require enrollment ...,-122.416134,37.739223
5,6,Bernal Heights Neighborhood Center,Open,Prepared Meals (Free or Low-Cost),Chilled meal (single),Pickup,Seniors,Age 60+,515 Cortland Ave,Monday - Friday,...,(415) 206-2142,None,On Lok Day Services,Cuisine Type: American,1.585181e+12,94110,"Community dining programs, funded by the Depar...",Seniors (age 60+) can visit an individual site...,-122.416134,37.739223
6,7,Bernal Heights Recreation Center,Closed to public,Prepared Meals (Free or Low-Cost),Meals and snacks,On-Site,Children and Youth,Participant at DSW/Essential/Healthcare worker,500 Moultrie St,Daily,...,None,None,None,None,NaN,94110,DCYF is sponsoring daily meals and snacks at e...,Meals are provided during the day to children ...,-122.416109,37.738311
7,8,Bessie Carmichael Elementary School - Transiti...,Open,Groceries (Free or Low-Cost),Pop-up Pantry,Pickup,Low-income San Franciscans,Low-Income,375 7th St,Thursday,...,(415) 282-1900,http://FoodLocator.SFMFoodBank.org,SF Marin Food Bank,None,1.589933e+12,94103,"Food pantry programs distribute fresh produce,...",Regular/pre-existing sites require enrollment ...,-122.406361,37.776234
8,9,Bethel Christian Church,Inactive,Groceries (Free or Low-Cost),Food Pantry,Pickup,Low-income San Franciscans,Low-Income,1325 Valencia St,Saturday,...,(415) 282-1900,http://FoodLocator.SFMFoodBank.org,SF Marin Food Bank,None,1.589933e+12,94110,"Food pantry programs distribute fresh produce,...",Regular/pre-existing sites require enrollment ...,-122.420292,37.751714
9,10,Betty Ann Ong Recreation Center*,Closed to public,Prepared Meals (Free or Low-Cost),Meals and snacks,On-Site,Children and Youth,Participant at DSW/Essential/Healthcare worker,1199 Mason St,Daily,.

## Fix date updated
Convert from Unix timestamp to month-day-year format.

In [7]:
def convert_timestamp_to_date(timestamp):
  if not pd.isna(timestamp):
    stamp = int(timestamp/1000)
    return datetime.fromtimestamp(stamp).strftime("%m/%d/%y")
  else:
    return ''

df['date_updated'] = df.apply(lambda x: convert_timestamp_to_date(x['attributes.Date_updated']), axis=1)

In [8]:
today = date.today()

filename = 'sf_foodresources_raw_%s.csv' % (today.strftime("%m_%d_%Y"))
df.to_csv(filename)